In [16]:
import pandas as pd
import os
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.vecm import select_coint_rank
import datetime
import matplotlib
from statsmodels.tsa.stattools import adfuller
%matplotlib notebook

os.getcwd()

data = pd.read_csv('C:/Users/D110148/OneDrive - pzem/data/Cointegration_Test.csv', sep = ";")
data.index = [pd.to_datetime(i) for i in data['Index']]
data = data.iloc[:,1:]
data.where(data == 'NaN', np.nan)
data.interpolate(inplace=True)

data_to_coint = data[[data.columns[2],data.columns[4],data.columns[5]]]
results = coint_johansen(data_to_coint[data_to_coint.index > datetime.datetime(2021,6,23)], det_order = 0,k_ar_diff=2)

In [26]:
results.max_eig_stat, results.max_eig_stat_crit_vals

(array([12.0880841 ,  4.53049907,  0.60102007]),
 array([[18.8928, 21.1314, 25.865 ],
        [12.2971, 14.2639, 18.52  ],
        [ 2.7055,  3.8415,  6.6349]]))

In [18]:
results.trace_stat, results.trace_stat_crit_vals

array([[18.8928, 21.1314, 25.865 ],
       [12.2971, 14.2639, 18.52  ],
       [ 2.7055,  3.8415,  6.6349]])

In [19]:
results2=select_coint_rank(data_to_coint[data_to_coint.index > datetime.datetime(2021,6,23)], det_order = 0,k_ar_diff=2)

(array([17.21960324,  5.13151914,  0.60102007]),
 array([[27.0669, 29.7961, 35.4628],
        [13.4294, 15.4943, 19.9349],
        [ 2.7055,  3.8415,  6.6349]]))

In [27]:
results2.summary()

r_0,r_1,test statistic,critical value
0,3,17.22,29.80


In [28]:
results.eig

array([0.04594633, 0.01747393, 0.00233587])

In [32]:
all(results.cvm[0]) == all(results.max_eig_stat_crit_vals[0])

True

In [33]:
results.ind

array([0, 1, 2], dtype=int64)

In [36]:
results.max_eig_stat

array([12.0880841 ,  4.53049907,  0.60102007])